In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

# Importamos las librerías necesarias
import cv2 as cv
import numpy as np


Using matplotlib backend: Qt5Agg


In [2]:
def calculate_frames_median(cap, actual_frame, N = 30):
    
    # Set the capture in the acutal frame
    cap.set(cv.CAP_PROP_POS_FRAMES, actual_frame)
    # Randomly select N frames
    frame_ids = cap.get(cv.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=N)

    # Store selected frames in an array
    frames = []
    for fid in frame_ids:
        ret, frame = cap.read()
        frames.append(frame)

    # Calculate the median along the time axis

    return np.median(frames, axis=0).astype(dtype=np.uint8)    



In [3]:
# Método de sustracción de fondo basado en mezclas gausianas

#metodo = 'MOG2'
metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = False)

In [8]:
# Abrimos el archivo
#-------------------
filename = 'vtest.avi'
capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

#Start the median frame calculation with all the frames
grayMedianFrame = cv.cvtColor(calculate_frames_median(capture, 1, 30), cv.COLOR_BGR2GRAY)

# Counter to know the number of elapsed cycles
counter = 0
# Time elapsed in ms for each cycle
ms_to_wait = 30
# How many time (in ms) to recalculate the median frame
recal_time = 3000
# Loop over all frames
while(1):
  
  ret, frame = capture.read()
  
  if ret == True:
      counter += 1

      # Calculate foreground mask from the gaussian method
      fgMask = backSub.apply(frame)

      if counter%int(recal_time/ms_to_wait) == 0:
        # Cut the median frame calculation >= actual frame
        actual_frame = capture.get(cv.CAP_PROP_POS_FRAMES)
        #print(actual_frame)
        grayMedianFrame = cv.cvtColor(calculate_frames_median(capture, actual_frame, 25), cv.COLOR_BGR2GRAY)
        
      # Convert current frame to grayscale
      grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
      # Calculate absolute difference of current frame and the median frame
      frame = cv.absdiff(grayImage, grayMedianFrame)
      # Otsu binarization
      used_tresh, frame = cv.threshold(frame,30,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

      # FG Mask from Naive-median method vs Gaussian
      cv.imshow('FG Mask Naive-median', frame)
      cv.imshow('FG Mask Gaussian', fgMask)

      k = cv.waitKey(ms_to_wait) & 0xff
      if k == ord("q"):
        break
  else:
      break

# Destroy all windows
cv.destroyAllWindows()

# Release video object
capture.release()



### Diferencias con método basado en mezcla gaussiana

Al comparar los vídeos, vemos que el método de sustracción de fondo naive usando la mediana como estimador genera mucho menos ruido (a excepción de un error de implementación al actualizar los frames de mi parte) comparado con los métodos basados en mezclas gaussianas. Sin embargo, algo que se puede apreciar, es que la segmentación hecha de las personas y objetos que se mueven es mucho mejor con los métodos basados en mezclas gaussianas, realizando un postprocesamiento adecuado del ruido generado, podría dar mejores resultados comparado con el de naive.

![Naive vs Gaussian](evidence.png)